In [1]:
# 1. Import library
import numpy as np
from pprint import pprint

In [2]:
## 2. Param
K = 2

In [3]:
# 3. Document vector
data = [
    [2.25, 0.01],
    [0.37, 2.93],
    [1.84, 0.14],
    [0.23, 2.54],
]
# data = [
#     [0, 0],
#     [0, 1],
#     [1, 1],
#     [3, 0],
# ]
idx_ignore = []

In [4]:
# 4. Distance function
def euclidian_dist(p1, p2): # point 1, point 2
    x = np.absolute(p1[0] - p2[0])
    y = np.absolute(p1[1] - p2[1])

    x2 = x ** 2
    y2 = y ** 2

    dist = np.sqrt(x2 + y2)

    return dist

def manhattan_dist(p1, p2): # point 1, point 2
    x = np.absolute(p1[0] - p2[0])
    y = np.absolute(p1[1] - p2[1])

    dist = x + y

    return dist

In [5]:
dist_dict = {}

for i in range(len(data)):
    for j in range(i+1, len(data)):
        dist_dict[f'd{i}_d{j}'] = euclidian_dist(data[i], data[j])

dist_dict

{'d0_d1': 3.4728662513837185,
 'd0_d2': 0.43011626335213127,
 'd0_d3': 3.2374835906920056,
 'd1_d2': 3.1535694062442956,
 'd1_d3': 0.41436698710201336,
 'd2_d3': 2.89}

In [6]:
def get_kth_dist(p):
    dists = []
    for i, d in enumerate(data):
        if d != p and i not in idx_ignore:
            dist = euclidian_dist(d, p)
            dists.append((dist, i))
    sorted_dists = dists.copy()
    sorted_dists.sort()
    nn_index = []
    for i in range(K):
        nn_index.append(
            sorted_dists[:][i][1]
        )

    return sorted_dists[:][K-1][0], nn_index

kth_dist_dict = {}
kth_nn_dict = {}
for i, d1 in enumerate(data):
    dist, nn = get_kth_dist(d1)
    kth_dist_dict[f'd{i}'] = dist
    kth_nn_dict[f'd{i}'] = nn
pprint(kth_dist_dict)
pprint(kth_nn_dict)

{'d0': 3.2374835906920056, 'd1': 3.1535694062442956, 'd2': 2.89, 'd3': 2.89}
{'d0': [2, 3], 'd1': [3, 2], 'd2': [0, 3], 'd3': [1, 2]}


In [7]:
lrd_dict = {}

for i, _ in enumerate(data):
    reach_distance = []
    for idx in kth_nn_dict[f'd{i}']:
        if f'd{i}_d{idx}' in dist_dict:
            reach_distance.append(max(
                kth_dist_dict[f'd{idx}'], dist_dict[f'd{i}_d{idx}']
            ))
        else:
            reach_distance.append(max(
                kth_dist_dict[f'd{idx}'], dist_dict[f'd{idx}_d{i}']
            ))
    lrd = K / sum(reach_distance)
    lrd_dict[f'd{i}'] = lrd

lrd_dict

{'d0': 0.32639826290813956,
 'd1': 0.3309302608378376,
 'd2': 0.32639826290813956,
 'd3': 0.3309302608378376}

In [8]:
lof_dict = {}

for i, _ in enumerate(data):
    lrd = []
    reach_distance = []
    for idx in kth_nn_dict[f'd{i}']:
        lrd.append(lrd_dict[f'd{idx}'])
        if f'd{i}_d{idx}' in dist_dict:
            reach_distance.append(max(
                kth_dist_dict[f'd{idx}'], dist_dict[f'd{i}_d{idx}']
            ))
        else:
            reach_distance.append(max(
                kth_dist_dict[f'd{idx}'], dist_dict[f'd{idx}_d{i}']
            ))

    lof = sum(lrd) * sum(reach_distance) / (K * K)

    lof_dict[f'd{i}'] = lof
lof_dict

{'d0': 1.0069424357368189,
 'd1': 0.9931526389907287,
 'd2': 1.0069424357368189,
 'd3': 0.9931526389907287}